In [1]:
import pandas as pd
import numpy as np

In [2]:
import chardet

# Read a small portion of the file to detect encoding
with open('data.csv', 'rb') as file:
    result = chardet.detect(file.read(100000))  # Read the first 100,000 bytes

# Print detected encoding
print(result)




{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [3]:
df=pd.read_csv('data.csv',encoding='latin1')

In [4]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [5]:
# Grouping by InvoiceNo and CustomerID, and joining the Description values
bills = df.groupby(['InvoiceNo', 'CustomerID'], as_index=False).agg({
    'Description': lambda x: ','.join(x)
})

# Rename the Description column if necessary
bills.rename(columns={'Description': 'Items'}, inplace=True)

In [6]:
bills

,InvoiceNo,CustomerID,Items
0,536365,17850.0,"WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL..."
1,536366,17850.0,"HAND WARMER UNION JACK,HAND WARMER RED POLKA DOT"
2,536367,13047.0,"ASSORTED COLOUR BIRD ORNAMENT,POPPY'S PLAYHOUS..."
3,536368,13047.0,"JAM MAKING SET WITH JARS,RED COAT RACK PARIS F..."
4,536369,13047.0,BATH BUILDING BLOCK WORD
...,...,...,...
22185,C581484,16446.0,"PAPER CRAFT , LITTLE BIRDIE"
22186,C581490,14397.0,"VICTORIAN GLASS HANGING T-LIGHT,ZINC T-LIGHT H..."
22187,C581499,15498.0,Manual
22188,C581568,15311.0,VICTORIAN SEWING BOX LARGE


In [7]:
# Replace '+' with 'and' in the 'Product' column
bills['Items'] = bills['Items'].str.replace('+', ' with ', regex=False)

In [8]:
# Creating a new column that counts the total items in the 'Items' column
bills['TotalItems'] = bills['Items'].apply(lambda x: len(x.split(',')))

In [9]:
bills

,InvoiceNo,CustomerID,Items,TotalItems
0,536365,17850.0,"WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL...",7
1,536366,17850.0,"HAND WARMER UNION JACK,HAND WARMER RED POLKA DOT",2
2,536367,13047.0,"ASSORTED COLOUR BIRD ORNAMENT,POPPY'S PLAYHOUS...",12
3,536368,13047.0,"JAM MAKING SET WITH JARS,RED COAT RACK PARIS F...",4
4,536369,13047.0,BATH BUILDING BLOCK WORD,1
...,...,...,...,...
22185,C581484,16446.0,"PAPER CRAFT , LITTLE BIRDIE",2
22186,C581490,14397.0,"VICTORIAN GLASS HANGING T-LIGHT,ZINC T-LIGHT H...",2
22187,C581499,15498.0,Manual,1
22188,C581568,15311.0,VICTORIAN SEWING BOX LARGE,1


In [10]:
# Split the entries by commas and flatten the list
all_items = bills['Items'].str.split(',').explode()

# Get unique items
unique_items = all_items.unique()

# Create a DataFrame with unique items
items = pd.DataFrame(unique_items, columns=['Unique Products'])

In [11]:
items

,Unique Products
0,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART.
...,...
3938,CREAM SWEETHEART TRAYS
3939,BLUE FLYING SINGING CANARY
3940,SWEETHEART KEY CABINET
3941,SMALL TAHITI BEACH BAG


In [12]:
# Create an empty list to store the results
dataset = []

# Iterate through each unique product
for product in items['Unique Products']:
    # Find rows in 'bills' where 'Items' contain the product
    matching_invoices = bills[bills['Items'].str.contains(product, na=False)]['CustomerID']
    
    # Combine all matching invoice numbers into a single string separated by commas
    custid = ', '.join(matching_invoices.astype(str))
    
    # Append the product and its matching invoice numbers to the results list
    dataset.append([product, custid])

# Create a DataFrame with the results
dataset = pd.DataFrame(dataset, columns=['Product', 'CustomerID'])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6628\2373009931.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matching_invoices = bills[bills['Items'].str.contains(product, na=False)]['CustomerID']


In [13]:
dataset

,Product,CustomerID
0,WHITE HANGING HEART T-LIGHT HOLDER,"17850.0, 17850.0, 17850.0, 17511.0, 13408.0, 1..."
1,WHITE METAL LANTERN,"17850.0, 17850.0, 17850.0, 17850.0, 17850.0, 1..."
2,CREAM CUPID HEARTS COAT HANGER,"17850.0, 17850.0, 17850.0, 17850.0, 17850.0, 1..."
3,KNITTED UNION FLAG HOT WATER BOTTLE,"17850.0, 17850.0, 17850.0, 17511.0, 17850.0, 1..."
4,RED WOOLLY HOTTIE WHITE HEART.,"17850.0, 17850.0, 17850.0, 17850.0, 17850.0, 1..."
...,...,...
3938,CREAM SWEETHEART TRAYS,13113.0
3939,BLUE FLYING SINGING CANARY,14800.0
3940,SWEETHEART KEY CABINET,13113.0
3941,SMALL TAHITI BEACH BAG,15299.0


In [14]:
dataset['CustomerID'] = dataset['CustomerID'].str.replace('.0','', regex=False)

In [15]:
# Apply the transformation to the entire 'CustomerID' column, filtering out empty strings
dataset['CustomerID'] = dataset['CustomerID'].apply(
    lambda x: [int(i.strip()) for i in x.split(',') if i.strip() != '']
)


In [16]:
# Remove duplicates from each row of the 'CustomerID' column
dataset['CustomerID'] = dataset['CustomerID'].apply(lambda x: list(set(x)))


In [17]:
# Creating a new column that counts the total items in the 'Items' column
dataset['TotalItems'] = dataset['CustomerID'].apply(lambda x: len(x))

In [18]:
dataset=dataset.sort_values(by='TotalItems', ascending=False)

In [19]:
dataset

,Product,CustomerID,TotalItems
1324,,"[16384, 16385, 16386, 16387, 16389, 16392, 163...",4372
190,,"[16384, 16385, 16386, 16387, 16389, 16392, 163...",4363
840,B,"[16384, 16385, 16386, 16387, 16389, 16392, 163...",4229
1626,PINK,"[16384, 16385, 16386, 16387, 16389, 16392, 163...",2904
3571,BLUE,"[16384, 16385, 16386, 16392, 16393, 16394, 163...",2573
...,...,...,...
2420,GLASS BEAD HOOP EARRINGS AMETHYST,[15279],1
3579,MINT DINER CLOCK,[14211],1
2443,PURPLE FRANGIPANI HAIRCLIP,[17961],1
3942,CRUK Commission,[14096],1


In [20]:
dataset = dataset.drop(index=1324)

In [21]:
dataset = dataset.drop(index=190)

In [22]:
dataset.drop_duplicates(subset=['Product'],inplace=True)

In [23]:
dataset = dataset.drop(index=840)

In [24]:
dataset

,Product,CustomerID,TotalItems
1626,PINK,"[16384, 16385, 16386, 16387, 16389, 16392, 163...",2904
3571,BLUE,"[16384, 16385, 16386, 16392, 16393, 16394, 163...",2573
2708,PINK,"[16384, 16385, 16386, 16387, 16389, 16392, 163...",2508
1074,HANGING,"[16384, 16389, 16394, 16395, 16399, 16400, 164...",2092
2402,METAL,"[16384, 16385, 16386, 16389, 16393, 16394, 163...",1978
...,...,...,...
2420,GLASS BEAD HOOP EARRINGS AMETHYST,[15279],1
3579,MINT DINER CLOCK,[14211],1
2443,PURPLE FRANGIPANI HAIRCLIP,[17961],1
3942,CRUK Commission,[14096],1


In [25]:
dataset.to_csv('Product.csv',index=False)